In [43]:
import $ivy.`org.apache.spark::spark-sql:2.4.5` 
import $ivy.`sh.almond::almond-spark:0.4.0`

import org.apache.spark.sql.{NotebookSparkSession, SparkSession}
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.types._

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

Logger.getRootLogger().setLevel(Level.ERROR)

Creating SparkSession


Spark UI

import $ivy.$                                   

import $ivy.$                              


import org.apache.spark.sql.{NotebookSparkSession, SparkSession}

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.types._


spark: SparkSession = org.apache.spark.sql.SparkSession@185ef8e5
import spark.implicits._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}



In [44]:
spark.read.format("json").load("D:/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json").schema

load at cmd43.sc:1

2 / 2

load at cmd43.sc:1

2 / 2

res43: StructType = StructType(
  StructField("DEST_COUNTRY_NAME", StringType, true, {}),
  StructField("ORIGIN_COUNTRY_NAME", StringType, true, {}),
  StructField("count", LongType, true, {})
)

In [45]:
import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}
import org.apache.spark.sql.types.Metadata

val myManualSchema = StructType(Array(
  StructField("DEST_COUNTRY_NAME", StringType, true),
  StructField("ORIGIN_COUNTRY_NAME", StringType, true),
  StructField("count", LongType, false,
    Metadata.fromJson("{\"hello\":\"world\"}"))
))

val df = spark.read.format("json").schema(myManualSchema)
  .load("D:/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json")

import org.apache.spark.sql.types.{StructField, StructType, StringType, LongType}

import org.apache.spark.sql.types.Metadata


myManualSchema: StructType = StructType(
  StructField("DEST_COUNTRY_NAME", StringType, true, {}),
  StructField("ORIGIN_COUNTRY_NAME", StringType, true, {}),
  StructField("count", LongType, false, {"hello":"world"})
)
df: DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [46]:
df.orderBy($"count".asc).show

show at cmd45.sc:1

1 / 1

show at cmd45.sc:1

1 / 1

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Estonia|    1|
|              Kosovo|      United States|    1|
|              Zambia|      United States|    1|
|       United States|   Papua New Guinea|    1|
|               Malta|      United States|    1|
|       United States|          Gibraltar|    1|
|            Suriname|      United States|    1|
|       United States|            Croatia|    1|
|            Djibouti|      United States|    1|
|        Burkina Faso|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|             Cyprus|    1|
|       United States|          Singapore|    1|
|             Moldova|      United States|    1|
|              Cyprus|      United States|    1|
|       United States|          Lithuania|    1|
|       United States|           Bulgaria|    1|
|       United State

In [47]:
df.col("count")

res46: Column = count

In [48]:
import org.apache.spark.sql.functions.expr
expr("(((someCol + 5) * 200) - 6) < otherCol")

import org.apache.spark.sql.functions.expr

res47_1: Column = ((((someCol + 5) * 200) - 6) < otherCol)

In [49]:
df.columns

res48: Array[String] = Array(
  "DEST_COUNTRY_NAME",
  "ORIGIN_COUNTRY_NAME",
  "count"
)

In [50]:
df.first

first at cmd49.sc:1

1 / 1

first at cmd49.sc:1

1 / 1

res49: Row = [United States,Romania,15]

In [51]:
//import org.apache.spark.sql.Row
val myRow = Row("Hello", null, 1, false)

myRow: Row = [Hello,null,1,false]

In [52]:
val df = spark.read.format("json")
  .load("D:/Spark-The-Definitive-Guide-master/data/flight-data/json/2015-summary.json")
df.createOrReplaceTempView("dfTable")

load at cmd51.sc:2

2 / 2

load at cmd51.sc:2

2 / 2

df: DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [53]:
val myManualSchema = new StructType(Array(
  new StructField("some", StringType, true),
  new StructField("col", StringType, true),
  new StructField("names", LongType, false)))
val myRows = Seq(Row("Hello", null, 1L))
val myRDD = spark.sparkContext.parallelize(myRows)
val myDf = spark.createDataFrame(myRDD, myManualSchema)
myDf.show()

show at cmd52.sc:8

1 / 1

show at cmd52.sc:8

1 / 1

show at cmd52.sc:8

...

3 / 3

show at cmd52.sc:8

3 / 3

+-----+----+-----+
| some| col|names|
+-----+----+-----+
|Hello|null|    1|
+-----+----+-----+



myManualSchema: StructType = StructType(
  StructField("some", StringType, true, {}),
  StructField("col", StringType, true, {}),
  StructField("names", LongType, false, {})
)
myRows: Seq[Row] = List([Hello,null,1])
myRDD: org.apache.spark.rdd.RDD[Row] = ParallelCollectionRDD[174] at parallelize at cmd52.sc:6
myDf: DataFrame = [some: string, col: string ... 1 more field]

In [54]:
df.select("DEST_COUNTRY_NAME").show(2)

show at cmd53.sc:1

1 / 1

show at cmd53.sc:1

1 / 1

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [55]:
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME"))
  .show(2)

show at cmd54.sc:2

1 / 1

show at cmd54.sc:2

1 / 1

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [56]:
df.selectExpr(
    "*", // include all original columns
    "(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")
  .show(2)

show at cmd55.sc:4

1 / 1

show at cmd55.sc:4

1 / 1

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [57]:
import org.apache.spark.sql.functions.lit
df.select(expr("*"), lit(1).as("One")).show(2)

show at cmd56.sc:2

1 / 1

show at cmd56.sc:2

1 / 1

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



import org.apache.spark.sql.functions.lit


In [58]:
df.withColumn("numberOne", lit(1)).show(2)

show at cmd57.sc:1

1 / 1

show at cmd57.sc:1

1 / 1

+-----------------+-------------------+-----+---------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|numberOne|
+-----------------+-------------------+-----+---------+
|    United States|            Romania|   15|        1|
|    United States|            Croatia|    1|        1|
+-----------------+-------------------+-----+---------+
only showing top 2 rows



In [59]:
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME"))
  .show(2)

show at cmd58.sc:2

1 / 1

show at cmd58.sc:2

1 / 1

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [60]:
df.withColumn("count2", $"count".cast("long"))

res59: DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 2 more fields]

In [61]:
import org.apache.spark.sql.functions.{col, column}
//df.filter(col("count") < 2).show(2)
df.filter($"count" < 2).show(2)
//df.where("count < 2").show(2)
df.where($"count" < 2).show(2)

show at cmd60.sc:3

1 / 1

show at cmd60.sc:3

1 / 1

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



...

1 / 1

show at cmd60.sc:5

1 / 1

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows



import org.apache.spark.sql.functions.{col, column}
//df.filter(col("count") < 2).show(2)


In [62]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()

count at cmd61.sc:1

1 / 1

count at cmd61.sc:1

1 / 1

count at cmd61.sc:1

...

count at cmd61.sc:1

1 / 1

count at cmd61.sc:1

1 / 1

res61: Long = 256L

In [63]:
val seed = 5
val withReplacement = false
val fraction = 0.5
df.sample(withReplacement, fraction, seed).count()

count at cmd62.sc:4

1 / 1

count at cmd62.sc:4

1 / 1

count at cmd62.sc:4

1 / 1

count at cmd62.sc:4

1 / 1

seed: Int = 5
withReplacement: Boolean = false
fraction: Double = 0.5
res62_3: Long = 126L

In [64]:
val schema = df.schema
val newRows = Seq(
  Row("New Country", "Other Country", 5L),
  Row("New Country 2", "Other Country 3", 1L)
)
val parallelizedRows = spark.sparkContext.parallelize(newRows)
val newDF = spark.createDataFrame(parallelizedRows, schema)
newDF.show()

show at cmd63.sc:8

1 / 1

show at cmd63.sc:8

1 / 1

show at cmd63.sc:8

...

3 / 3

show at cmd63.sc:8

3 / 3

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|      New Country|      Other Country|    5|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



schema: StructType = StructType(
  StructField("DEST_COUNTRY_NAME", StringType, true, {}),
  StructField("ORIGIN_COUNTRY_NAME", StringType, true, {}),
  StructField("count", LongType, true, {})
)
newRows: Seq[Row] = List(
  [New Country,Other Country,5],
  [New Country 2,Other Country 3,1]
)
parallelizedRows: org.apache.spark.rdd.RDD[Row] = ParallelCollectionRDD[227] at parallelize at cmd63.sc:6
newDF: DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [65]:
df.union(newDF)
  .where("count = 1")
  .where($"ORIGIN_COUNTRY_NAME" =!= "United States")
  .show()

show at cmd64.sc:3

1 / 1

show at cmd64.sc:3

1 / 1

show at cmd64.sc:3

...

show at cmd64.sc:3

2 / 2

show at cmd64.sc:3

2 / 2

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



In [66]:
df.limit(5).show()

show at cmd65.sc:1

1 / 1

show at cmd65.sc:1

1 / 1

show at cmd65.sc:1

1 / 1

show at cmd65.sc:1

1 / 1

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [67]:
df.rdd.getNumPartitions

res66: Int = 1

In [68]:
val collectDF = df.limit(10)
collectDF.take(5) // take works with an Integer count

take at cmd67.sc:2

1 / 1

take at cmd67.sc:2

1 / 1

collectDF: Dataset[Row] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]
res67_1: Array[Row] = Array(
  [United States,Romania,15],
  [United States,Croatia,1],
  [United States,Ireland,344],
  [Egypt,United States,15],
  [United States,India,62]
)

In [69]:
collectDF.show() // this prints it out nicely

show at cmd68.sc:1

1 / 1

show at cmd68.sc:1

1 / 1

show at cmd68.sc:1

1 / 1

show at cmd68.sc:1

1 / 1

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+



In [70]:
collectDF.show(5, false)

show at cmd69.sc:1

1 / 1

show at cmd69.sc:1

1 / 1

show at cmd69.sc:1

1 / 1

show at cmd69.sc:1

1 / 1

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|United States    |Romania            |15   |
|United States    |Croatia            |1    |
|United States    |Ireland            |344  |
|Egypt            |United States      |15   |
|United States    |India              |62   |
+-----------------+-------------------+-----+
only showing top 5 rows



In [71]:
collectDF.collect()

collect at cmd70.sc:1

1 / 1

collect at cmd70.sc:1

1 / 1

res70: Array[Row] = Array(
  [United States,Romania,15],
  [United States,Croatia,1],
  [United States,Ireland,344],
  [Egypt,United States,15],
  [United States,India,62],
  [United States,Singapore,1],
  [United States,Grenada,62],
  [Costa Rica,United States,588],
  [Senegal,United States,40],
  [Moldova,United States,1]
)

## Capitulo 6

In [72]:
val df = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("D:/Spark-The-Definitive-Guide-master/data/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")    //creará una vista temporal de la tabla en la memoria

load at cmd71.sc:4

1 / 1

load at cmd71.sc:4

1 / 1

load at cmd71.sc:4

2 / 2

load at cmd71.sc:4

2 / 2

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



df: DataFrame = [InvoiceNo: string, StockCode: string ... 6 more fields]

In [73]:
import org.apache.spark.sql.functions.lit
df.select(lit(5), lit("five"), lit(5.0))

import org.apache.spark.sql.functions.lit

res72_1: DataFrame = [5: int, five: string ... 1 more field]

In [74]:
df.where(col("InvoiceNo").equalTo(536365))
  .select("InvoiceNo", "Description")
  .show(5, false)

show at cmd73.sc:3

1 / 1

show at cmd73.sc:3

1 / 1

+---------+-----------------------------------+
|InvoiceNo|Description                        |
+---------+-----------------------------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |
|536365   |WHITE METAL LANTERN                |
|536365   |CREAM CUPID HEARTS COAT HANGER     |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |
+---------+-----------------------------------+
only showing top 5 rows



Lo mismo

In [75]:
df.where("InvoiceNo = 536365")
  .show(5, false)

show at cmd74.sc:2

1 / 1

show at cmd74.sc:2

1 / 1

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [78]:
val priceFilter = col("UnitPrice") > 600
val descripFilter = col("Description").contains("POSTAGE")
df.where(col("StockCode").isin("DOT")).where(priceFilter.or(descripFilter))
  .show()

show at cmd77.sc:3

1 / 1

show at cmd77.sc:3

1 / 1

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      null|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



priceFilter: Column = (UnitPrice > 600)
descripFilter: Column = contains(Description, POSTAGE)

In [84]:
val DOTCodeFilter = col("StockCode") === "DOT"
val priceFilter = col("UnitPrice") > 600
val descripFilter = col("Description").contains("POSTAGE")
df.withColumn("isExpensive", DOTCodeFilter.and(priceFilter.or(descripFilter)))
  .where("isExpensive")
  //.filter("isExpensive")
  .select("unitPrice", "isExpensive").show(5)

show at cmd83.sc:7

1 / 1

show at cmd83.sc:7

1 / 1

+---------+-----------+
|unitPrice|isExpensive|
+---------+-----------+
|   569.77|       true|
|   607.49|       true|
+---------+-----------+



DOTCodeFilter: Column = (StockCode = DOT)
priceFilter: Column = (UnitPrice > 600)
descripFilter: Column = contains(Description, POSTAGE)

In [85]:
import org.apache.spark.sql.functions.{expr, not, col}
df.withColumn("isExpensive", not(col("UnitPrice").leq(250)))
  .filter("isExpensive")
  .select("Description", "UnitPrice").show(5)
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))
  .filter("isExpensive")
  .select("Description", "UnitPrice").show(5)

show at cmd84.sc:4

1 / 1

show at cmd84.sc:4

1 / 1

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



...

1 / 1

show at cmd84.sc:7

1 / 1

+--------------+---------+
|   Description|UnitPrice|
+--------------+---------+
|DOTCOM POSTAGE|   569.77|
|DOTCOM POSTAGE|   607.49|
+--------------+---------+



import org.apache.spark.sql.functions.{expr, not, col}


In [87]:
df.where(col("Description").eqNullSafe("CustumerID")).show()

show at cmd86.sc:1

1 / 1

show at cmd86.sc:1

1 / 1

+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
+---------+---------+-----------+--------+-----------+---------+----------+-------+

